# Eff. matching to GEN particle


   * [electrons 2 < pt < 200 GeV](#Electron-sample-pt-flat-2---100-GeV)
   * [photons pt 35GeV 1.6<|eta|<2.8](#Photon-pt35-GeV-1.6-<-|eta|-<-2.8)


In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)


# define some utility functions
def newCanvas(name=None, title=None, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title
    canvas = ROOT.TCanvas(name, title)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def draw(plot, options='', text=None):
    c = newCanvas()
    c.cd()
    plot.Draw(options)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()

    return


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


class Sample():
    def __init__(cls, name, label, version=None):
        cls.name = name
        cls.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        cls.histo_filename = '../plots1/histos_{}{}.root'.format(cls.name, version)
        cls.histo_file = ROOT.TFile(cls.histo_filename)


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def drawSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2

    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        hist.SetStats(False)
        if norm:
            hist.DrawNormalized('same'+','+options, 1.)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()


def drawProfileX(histograms, labels, options=''):
    profiles = [hist.ProfileX() for hist in histograms]
    drawSame(profiles, labels, options)


def drawSeveral(histograms, labels, options='', do_profile=False, miny=None, maxy=None, text=None):
    ydiv = int(math.ceil(float(len(histograms))/2))
    for hidx in range(0, len(histograms)):
        newtext = labels[hidx]
        if text:
            newtext = '{}: {}'.format(labels[hidx], text)
        if do_profile:
            drawAndProfileX(histograms[hidx], miny=miny, maxy=maxy, options=options, do_profile=do_profile, text=newtext)
        else:
            draw(histograms[hidx], options=options, text=newtext)


def drawProfileRatio(prof1, prof2, ymin=None, ymax=None, text=None):
    hist1 = prof1.ProjectionX(uuid.uuid4().hex[:6])
    hist2 = prof2.ProjectionX(uuid.uuid4().hex[:6])
    hist1.Divide(hist2)
    draw(hist1)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if ymin is not None and ymax is not None:
        hist1.GetYaxis().SetRangeUser(ymin, ymax)
    ROOT.gPad.Update()

   
# mean+-nsigmas*RMS.
def drawGaussFit(histo, nsigmas, min, max):
    minfit = histo.GetMean() - nsigmas*histo.GetRMS()
    maxfit = histo.GetMean() + nsigmas*histo.GetRMS()
    drawGFit(histo, min, max, minfit, maxfit)


# Fit a histogram in the range (minfit, maxfit) with a gaussian and
# draw it in the range (min, max)
def drawGFit(histo, min, max, minfit, maxfit):
    # static int i = 0
    # i++
    # gPad->SetGrid(1,1);
    # gStyle->SetGridColor(15);
    histo.GetXaxis().SetRangeUser(min,max)
    global f_idx
    nameF1 = "g{}".format(f_idx)
    f_idx +=1
    g1 = ROOT.TF1(nameF1,"gaus",minfit,maxfit)
    g1.SetLineColor(2)
    g1.SetLineWidth(2)
    histo.Fit(g1,"R")



def drawGraphsSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+','+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    if min_y is not None and max_y is not None:
        histograms[0].GetYaxis().SetRangeUser(min_y, max_y)
        
    leg.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    c.Draw()



In [3]:
version = 'v35'

#      Particle('ele', PID.electron),
#                  Particle('ele_A', PID.electron, '1.4 < abseta < 1.7'),
#                  Particle('ele_B', PID.electron, '1.7 <= abseta <= 2.8'),
#                  Particle('ele_C', PID.electron, 'abseta > 2.8'),
#                  Particle('photon', PID.photon),
#                  Particle('photon_A', PID.photon, '1.4 < abseta < 1.7'),
#                  Particle('photon_B', PID.photon, '1.7 <= abseta <= 2.8'),
#                  Particle('photon_C', PID.photon, 'abseta > 2.8'),
                
particles = ''

sample_names = ['ele_flat2to100_PU0',
                'ele_flat2to100_PU200',
                'photonPt35_PU0',
                'photonPt35_PU200']




sample_ele_flat2to100_PU0 = Sample('ele_flat2to100_PU0', 'PU0', version)
sample_ele_flat2to100_PU200 = Sample('ele_flat2to100_PU200', 'PU200', version)

sample_gPt35_PU0 = Sample('photonPt35_PU0', 'Pt35 PU0', version)
sample_gPt35_PU200 = Sample('photonPt35_PU200', 'Pt35 PU200', version)

sample_hadronGun_PU0 = Sample('hadronGun_PU0', 'PU0', version)
sample_hadronGun_PU200 = Sample('hadronGun_PU200', 'PU200', version)


samples_ele = [sample_ele_flat2to100_PU0,sample_ele_flat2to100_PU200]
samples_ele = [sample_ele_flat2to100_PU200]

samples_photon = [sample_gPt35_PU0,sample_gPt35_PU200]
samples_hadrons = [sample_hadronGun_PU0,sample_hadronGun_PU200]



samples = []





Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU0_v35.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU0_v35.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU200_v35.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU0_v35.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU200_v35.root does not exist


## Electron sample pt flat 2 - 100 GeV

In [16]:

samples = samples_ele
tp_sets = ['DEF', 'DEF_em', 'DEF_pt20', 'DEF_pt20_em', 'DEF_pt25', 'DEF_pt25_em', 'DEF_pt30', 'DEF_pt30_em']
particles = ['ele', 'eleA', 'eleB', 'eleC', 'eleD']



tpset_labels = {'DEF' : 'NNDR',
                'DEF_em': 'NNDR + EGID',
                'DEF_pt20': 'NNDR, p_{T}>20GeV',
                'DEF_pt20_em': 'NNDR + EGID, p_{T}>20GeV',
                'DEF_pt25': 'NNDR, p_{T}>25GeV',
                'DEF_pt25_em': 'NNDR + EGID, p_{T}>25GeV',
                'DEF_pt30': 'NNDR, p_{T}>30GeV',
                'DEF_pt30_em': 'NNDR + EGID, p_{T}>30GeV'}


particle_labels = {'ele': 'all #eta',
                   'eleA' : '1.4 < |#eta| < 1.7',
                   'eleB' : '1.7 < |#eta| < 2.8',
                   'eleC' : '|#eta| > 2.8',
                   'eleD' : '|#eta| < 2.4',
                   'all'  : 'all #eta',
                   'etaA': '#eta < 1.7',
                   'etaB': '1.7 < |#eta| < 2.8)',
                   'etaC': '|#eta| > 2.8',
                   'etaD': '|#eta| < 2.4'}



labels = [sample.label for sample in samples]

histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)
titles_df = pd.DataFrame(index=tp_sets, columns=particles)
# print histo_num_df
for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        titles_df.loc[tp_set, particle] = '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])

histo_df



,ele,eleA,eleB,eleC,eleD
DEF,[<l1THistos.HistoEff instance at 0x11df55998>],[<l1THistos.HistoEff instance at 0x11df55ab8>],[<l1THistos.HistoEff instance at 0x11df559e0>],[<l1THistos.HistoEff instance at 0x11d925560>],[<l1THistos.HistoEff instance at 0x11d9315f0>]
DEF_em,[<l1THistos.HistoEff instance at 0x11dc35440>],[<l1THistos.HistoEff instance at 0x11dc35b90>],[<l1THistos.HistoEff instance at 0x11dc4bcf8>],[<l1THistos.HistoEff instance at 0x11dc4bb00>],[<l1THistos.HistoEff instance at 0x11dc4b758>]
DEF_pt20,[<l1THistos.HistoEff instance at 0x11df58b48>],[<l1THistos.HistoEff instance at 0x11df585f0>],[<l1THistos.HistoEff instance at 0x11df6b2d8>],[<l1THistos.HistoEff instance at 0x11df6b710>],[<l1THistos.HistoEff instance at 0x11df6b9e0>]
DEF_pt20_em,[<l1THistos.HistoEff instance at 0x11df6bcf8>],[<l1THistos.HistoEff instance at 0x11df6bea8>],[<l1THistos.HistoEff instance at 0x11dfe92d8>],[<l1THistos.HistoEff instance at 0x11dfe9560>],[<l1THistos.HistoEff instance at 0x11dfe97e8>]
DEF_pt25,[<l1THistos.HistoEff instance at 0x11dfe9998>],[<l1THistos.HistoEff instance at 0x11dfe9c68>],[<l1THistos.HistoEff instance at 0x11dfe9ef0>],[<l1THistos.HistoEff instance at 0x11dffc1b8>],[<l1THistos.HistoEff instance at 0x11dffc488>]
DEF_pt25_em,[<l1THistos.HistoEff instance at 0x11dffc7a0>],[<l1THistos.HistoEff instance at 0x11dffcab8>],[<l1THistos.HistoEff instance at 0x11dffcd40>],[<l1THistos.HistoEff instance at 0x11dffcfc8>],[<l1THistos.HistoEff instance at 0x11dfff290>]
DEF_pt30,[<l1THistos.HistoEff instance at 0x11dfff440>],[<l1THistos.HistoEff instance at 0x11dfff710>],[<l1THistos.HistoEff instance at 0x11dfff998>],[<l1THistos.HistoEff instance at 0x11dfffc20>],[<l1THistos.HistoEff instance at 0x11dfffef0>]
DEF_pt30_em,[<l1THistos.HistoEff instance at 0x11e004248>],[<l1THistos.HistoEff instance at 0x11e004560>],[<l1THistos.HistoEff instance at 0x11e0047e8>],[<l1THistos.HistoEff instance at 0x11e004a70>],[<l1THistos.HistoEff instance at 0x11e004cf8>]


In [5]:
titles_df

,ele,eleA,eleB,eleC,eleD
DEF,"NNDR, all #eta","NNDR, 1.4 < |#eta| < 1.7","NNDR, 1.7 < |#eta| < 1.8","NNDR, |#eta| > 2.8","NNDR, |#eta| < 2.4"
DEF_em,"NNDR + EGID, all #eta","NNDR + EGID, 1.4 < |#eta| < 1.7","NNDR + EGID, 1.7 < |#eta| < 1.8","NNDR + EGID, |#eta| > 2.8","NNDR + EGID, |#eta| < 2.4"
DEF_pt20,"NNDR, p_{T}>20GeV, all #eta","NNDR, p_{T}>20GeV, 1.4 < |#eta| < 1.7","NNDR, p_{T}>20GeV, 1.7 < |#eta| < 1.8","NNDR, p_{T}>20GeV, |#eta| > 2.8","NNDR, p_{T}>20GeV, |#eta| < 2.4"
DEF_pt20_em,"NNDR + EGID, p_{T}>20GeV, all #eta","NNDR + EGID, p_{T}>20GeV, 1.4 < |#eta| < 1.7","NNDR + EGID, p_{T}>20GeV, 1.7 < |#eta| < 1.8","NNDR + EGID, p_{T}>20GeV, |#eta| > 2.8","NNDR + EGID, p_{T}>20GeV, |#eta| < 2.4"
DEF_pt25,"NNDR, p_{T}>25GeV, all #eta","NNDR, p_{T}>25GeV, 1.4 < |#eta| < 1.7","NNDR, p_{T}>25GeV, 1.7 < |#eta| < 1.8","NNDR, p_{T}>25GeV, |#eta| > 2.8","NNDR, p_{T}>25GeV, |#eta| < 2.4"
DEF_pt25_em,"NNDR + EGID, p_{T}>25GeV, all #eta","NNDR + EGID, p_{T}>25GeV, 1.4 < |#eta| < 1.7","NNDR + EGID, p_{T}>25GeV, 1.7 < |#eta| < 1.8","NNDR + EGID, p_{T}>25GeV, |#eta| > 2.8","NNDR + EGID, p_{T}>25GeV, |#eta| < 2.4"
DEF_pt30,"NNDR, p_{T}>30GeV, all #eta","NNDR, p_{T}>30GeV, 1.4 < |#eta| < 1.7","NNDR, p_{T}>30GeV, 1.7 < |#eta| < 1.8","NNDR, p_{T}>30GeV, |#eta| > 2.8","NNDR, p_{T}>30GeV, |#eta| < 2.4"
DEF_pt30_em,"NNDR + EGID, p_{T}>30GeV, all #eta","NNDR + EGID, p_{T}>30GeV, 1.4 < |#eta| < 1.7","NNDR + EGID, p_{T}>30GeV, 1.7 < |#eta| < 1.8","NNDR + EGID, p_{T}>30GeV, |#eta| > 2.8","NNDR + EGID, p_{T}>30GeV, |#eta| < 2.4"


In [6]:
dir(histo_df.ele['DEF'][0])


['__doc__',
 '__init__',
 '__module__',
 'h_energy',
 'h_eta',
 'h_fBrem',
 'h_pt',
 'h_reachedEE']

In [7]:
histo_df.loc['DEF', 'ele']

[<l1THistos.HistoEff instance at 0x11d93aab8>]

In [8]:

for particle in ['ele']:
    for tp_set in tp_sets:
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.ele[tp_set]]
        drawGraphsSame(h_eff_vs_eta, labels, text=titles_df.loc[tp_set, particle])



In [9]:
for particle in particles:
    for tp_set in tp_sets:
        print 'particle: {}, TP set: {}, text: {}'.format(particle, tp_set, titles_df.loc[tp_set, particle])
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histo_df.loc[tp_set, particle]]
        drawGraphsSame(h_eff_vs_pt, labels, text=titles_df.loc[tp_set, particle], min_y=0., max_y=1.1)


particle: ele, TP set: DEF, text: NNDR, all #eta
particle: ele, TP set: DEF_em, text: NNDR + EGID, all #eta
particle: ele, TP set: DEF_pt20, text: NNDR, p_{T}>20GeV, all #eta
particle: ele, TP set: DEF_pt20_em, text: NNDR + EGID, p_{T}>20GeV, all #eta
particle: ele, TP set: DEF_pt25, text: NNDR, p_{T}>25GeV, all #eta
particle: ele, TP set: DEF_pt25_em, text: NNDR + EGID, p_{T}>25GeV, all #eta
particle: ele, TP set: DEF_pt30, text: NNDR, p_{T}>30GeV, all #eta
particle: ele, TP set: DEF_pt30_em, text: NNDR + EGID, p_{T}>30GeV, all #eta
particle: eleA, TP set: DEF, text: NNDR, 1.4 < |#eta| < 1.7
particle: eleA, TP set: DEF_em, text: NNDR + EGID, 1.4 < |#eta| < 1.7
particle: eleA, TP set: DEF_pt20, text: NNDR, p_{T}>20GeV, 1.4 < |#eta| < 1.7
particle: eleA, TP set: DEF_pt20_em, text: NNDR + EGID, p_{T}>20GeV, 1.4 < |#eta| < 1.7
particle: eleA, TP set: DEF_pt25, text: NNDR, p_{T}>25GeV, 1.4 < |#eta| < 1.7
particle: eleA, TP set: DEF_pt25_em, text: NNDR + EGID, p_{T}>25GeV, 1.4 < |#eta| < 1.

In [10]:
histo_den_df

,ele,eleA,eleB,eleC,eleD
DEF,[<GenParticleHistos h_effDen_DEF_ele>],[<GenParticleHistos h_effDen_DEF_eleA>],[<GenParticleHistos h_effDen_DEF_eleB>],[<GenParticleHistos h_effDen_DEF_eleC>],[<GenParticleHistos h_effDen_DEF_eleD>]
DEF_em,[<GenParticleHistos h_effDen_DEF_em_ele>],[<GenParticleHistos h_effDen_DEF_em_eleA>],[<GenParticleHistos h_effDen_DEF_em_eleB>],[<GenParticleHistos h_effDen_DEF_em_eleC>],[<GenParticleHistos h_effDen_DEF_em_eleD>]
DEF_pt20,[<GenParticleHistos h_effDen_DEF_pt20_ele>],[<GenParticleHistos h_effDen_DEF_pt20_eleA>],[<GenParticleHistos h_effDen_DEF_pt20_eleB>],[<GenParticleHistos h_effDen_DEF_pt20_eleC>],[<GenParticleHistos h_effDen_DEF_pt20_eleD>]
DEF_pt20_em,[<GenParticleHistos h_effDen_DEF_pt20_em_ele>],[<GenParticleHistos h_effDen_DEF_pt20_em_eleA>],[<GenParticleHistos h_effDen_DEF_pt20_em_eleB>],[<GenParticleHistos h_effDen_DEF_pt20_em_eleC>],[<GenParticleHistos h_effDen_DEF_pt20_em_eleD>]
DEF_pt25,[<GenParticleHistos h_effDen_DEF_pt25_ele>],[<GenParticleHistos h_effDen_DEF_pt25_eleA>],[<GenParticleHistos h_effDen_DEF_pt25_eleB>],[<GenParticleHistos h_effDen_DEF_pt25_eleC>],[<GenParticleHistos h_effDen_DEF_pt25_eleD>]
DEF_pt25_em,[<GenParticleHistos h_effDen_DEF_pt25_em_ele>],[<GenParticleHistos h_effDen_DEF_pt25_em_eleA>],[<GenParticleHistos h_effDen_DEF_pt25_em_eleB>],[<GenParticleHistos h_effDen_DEF_pt25_em_eleC>],[<GenParticleHistos h_effDen_DEF_pt25_em_eleD>]
DEF_pt30,[<GenParticleHistos h_effDen_DEF_pt30_ele>],[<GenParticleHistos h_effDen_DEF_pt30_eleA>],[<GenParticleHistos h_effDen_DEF_pt30_eleB>],[<GenParticleHistos h_effDen_DEF_pt30_eleC>],[<GenParticleHistos h_effDen_DEF_pt30_eleD>]
DEF_pt30_em,[<GenParticleHistos h_effDen_DEF_pt30_em_ele>],[<GenParticleHistos h_effDen_DEF_pt30_em_eleA>],[<GenParticleHistos h_effDen_DEF_pt30_em_eleB>],[<GenParticleHistos h_effDen_DEF_pt30_em_eleC>],[<GenParticleHistos h_effDen_DEF_pt30_em_eleD>]


In [14]:
# build turnon

for particle in particles:
    TP_set_den = 'DEF' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20', 'DEF_pt25', 'DEF_pt30'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

In [15]:
for particle in particles:
    TP_set_den = 'DEF_em' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

# Photon pt35 GeV 1.6 < |eta| < 2.8 

In [14]:

samples = samples_photon
tp_sets = ['DEF', 'DEF_em']
particles = ['photon']
labels = [sample.label for sample in samples]

histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)

for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        histo_df



KeyError: 'DEF'

In [ ]:
dir(histo_df.photon['DEF'][0])


In [ ]:
tp_set = 'DEF'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)

tp_set = 'DEF_em'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)
